# Objetivo do Código

Este código foi desenvolvido para dois objetivos principais:

1. **Explorar a distribuição da população urbana de São Paulo por bairros**:
   - Entender como a população está distribuída em diferentes regiões da cidade.

2. **Analisar o acesso ao transporte público**:
   - Identificar se os moradores têm um bom acesso aos ônibus oferecidos pela SPTrans.


In [2]:
# %pip install pandas
# %pip install requests  # Corrigido para 'requests'
# %pip install lxml
# !pip install requests


import pandas as pd
import requests

# URL da página com as tabelas
url = 'https://pt.wikipedia.org/wiki/Lista_dos_distritos_de_S%C3%A3o_Paulo_por_popula%C3%A7%C3%A3o'

# Baixar o conteúdo da página
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    tabelas = pd.read_html(response.text
                        #    , header=1
                           )  # Define a segunda linha como cabeçalho

    # Dicionário para armazenar os DataFrames
    dfs = {}

    # Criar um DataFrame para cada tabela e armazenar no dicionário
    for i, tabela in enumerate(tabelas):
        nome_df = f'df{i+1}'
        dfs[nome_df] = tabela
else:
    print(f"Erro ao acessar a página: {response.status_code}")

C:\Users\joth1\AppData\Local\Temp\ipykernel_24004\1303294201.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas = pd.read_html(response.text


In [3]:
# display(dfs['df2'])
df = dfs['df2'].copy()
df['Cidade'] = 'São Paulo'

df['População 2022'] = df['População 2022'].astype(str)
df['População 2022'] = df['População 2022'].str.replace('.', '')
df['População 2022'] = df['População 2022'].astype(int)

display(df)
display(df.dtypes)

,Posição,Distrito,População 2022,Cidade
0,1,Grajaú,384873,São Paulo
1,2,Jardim Ângela,311432,São Paulo
2,3,Capão Redondo,270767,São Paulo
3,4,Sapopemba,266715,São Paulo
4,5,Sacomã,261436,São Paulo
...,...,...,...,...
91,92,Barra Funda,33436,São Paulo
92,93,Jaguara,2473,São Paulo
93,94,Sé,23832,São Paulo
94,95,Pari,17359,São Paulo


Posição            int64
Distrito          object
População 2022     int32
Cidade            object
dtype: object

#### Numero de pessoas presentes na região urbana de São Paulo

In [4]:
cubo = df.groupby('Cidade').agg(
    qtd=('População 2022', 'sum')
)
cubo

,qtd
Cidade,
São Paulo,10259967


### Capturo a Região de cada um dos bairro ( Latitude E Longitude )

In [5]:
import requests

def get_coordinates_by_bairro(bairro):
    # URL para fazer a requisição à API Nominatim com o nome do bairro
    url = f'https://nominatim.openstreetmap.org/search?q={bairro}&format=json&addressdetails=1'
    
    # Cabeçalho 'User-Agent' necessário para evitar erro 403
    headers = {
        'User-Agent': 'Jonathan/joth10j@gmail.com'  # Substitua pelo seu nome e email ou o nome do seu projeto
    }
    
    # Fazendo a requisição
    response = requests.get(url, headers=headers)
    
    # Verifica se a resposta foi bem-sucedida (status code 200)
    if response.status_code == 200:
        try:
            data = response.json()  # Converte a resposta para JSON
            if data:
                # A primeira entrada da lista geralmente é a mais precisa
                result = data[0]  # Pega o primeiro resultado encontrado
                lat = result['lat']  # Latitude
                lon = result['lon']  # Longitude
                
                return lat, lon
            else:
                return "Bairro não encontrado"
        except ValueError:
            return "Erro ao decodificar a resposta JSON."
    elif response.status_code == 403:
        return "Erro 403: Acesso negado. Verifique as limitações da API."
    else:
        return f"Erro na requisição: {response.status_code}"


# Aplicando a função ao DataFrame
df['latitude_longitude'] = df['Distrito'].apply(get_coordinates_by_bairro)
df

#### EM PRODUÇÃO: Crio um padrão de reconhecimento de região ( Zona Leste, Oeste, Sul e Norte de São Paulo)

In [6]:

# Função para determinar a região com base em latitude e longitude
def determinar_regiao(lat, lon):
    # Zona Norte: Norte de São Paulo, áreas acima de -23.6 (latitude) e oeste de -46.7 (longitude)
    if lat >= -23.6 and lon <= -46.7:
        return "Zona Norte"
    
    # Zona Sul: Abaixo de -23.7 (latitude) e a oeste de -46.7 (longitude)
    elif lat <= -23.7 and lon <= -46.7:
        return "Zona Sul"
    
    # Zona Leste: Abaixo de -23.7 (latitude) e a leste de -46.6 (longitude)
    elif lat <= -23.7 and lon > -46.6:
        return "Zona Leste"
    
    # Zona Oeste: Acima de -23.6 (latitude) e a leste de -46.6 (longitude)
    elif lat >= -23.6 and lon > -46.6:
        return "Zona Oeste"
    
    # Centro: Área central da cidade, bem delimitada entre latitudes de -23.6 e -23.5 e longitudes entre -46.7 e -46.6
    elif -23.6 < lat < -23.5 and -46.7 < lon < -46.6:
        return "Centro"
    
    # Caso não se encaixe em nenhuma das regiões, retornamos "Região desconhecida"
    else:
        return "Região desconhecida"


# Função para extrair latitude e longitude das tuplas
def extrair_lat_lon(tupla):
    lat, lon = tupla
    return lat, lon

# Separando a tupla de latitude e longitude em duas colunas
df[['latitude', 'longitude']] = pd.DataFrame(df['latitude_longitude'].apply(extrair_lat_lon).tolist(), index=df.index)
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Aplicando a função de classificação de região
df['Regiao'] = df.apply(lambda row: determinar_regiao(row['latitude'], row['longitude']), axis=1)
df

,Posição,Distrito,População 2022,Cidade,latitude_longitude,latitude,longitude,Regiao
0,1,Grajaú,384873,São Paulo,"(-5.8154034, -46.1361454)",-5.815403,-46.136145,Zona Oeste
1,2,Jardim Ângela,311432,São Paulo,"(-23.7125278, -46.7687195)",-23.712528,-46.768720,Zona Sul
2,3,Capão Redondo,270767,São Paulo,"(-23.6719026, -46.779435420915036)",-23.671903,-46.779435,Região desconhecida
3,4,Sapopemba,266715,São Paulo,"(-23.6043265, -46.5098851)",-23.604326,-46.509885,Região desconhecida
4,5,Sacomã,261436,São Paulo,"(-23.6012824, -46.6025552)",-23.601282,-46.602555,Região desconhecida
...,...,...,...,...,...,...,...,...
91,92,Barra Funda,33436,São Paulo,"(-27.9230048, -53.0401045)",-27.923005,-53.040104,Zona Sul
92,93,Jaguara,2473,São Paulo,"(-12.10919355, -39.12136265382381)",-12.109194,-39.121363,Zona Oeste
93,94,Sé,23832,São Paulo,"(59.6749712, 14.5208584)",59.674971,14.520858,Zona Oeste
94,95,Pari,17359,São Paulo,"(48.8534951, 2.3483915)",48.853495,2.348391,Zona Oeste


In [7]:
df.head(20)

,Posição,Distrito,População 2022,Cidade,latitude_longitude,latitude,longitude,Regiao
0,1,Grajaú,384873,São Paulo,"(-5.8154034, -46.1361454)",-5.815403,-46.136145,Zona Oeste
1,2,Jardim Ângela,311432,São Paulo,"(-23.7125278, -46.7687195)",-23.712528,-46.768720,Zona Sul
2,3,Capão Redondo,270767,São Paulo,"(-23.6719026, -46.779435420915036)",-23.671903,-46.779435,Região desconhecida
3,4,Sapopemba,266715,São Paulo,"(-23.6043265, -46.5098851)",-23.604326,-46.509885,Região desconhecida
4,5,Sacomã,261436,São Paulo,"(-23.6012824, -46.6025552)",-23.601282,-46.602555,Região desconhecida
5,6,Jardim São Luís,259377,São Paulo,"(-23.6739274, -46.7407377)",-23.673927,-46.740738,Região desconhecida
6,7,Cidade Ademar,249218,São Paulo,"(-23.6730116, -46.6552806)",-23.673012,-46.655281,Região desconhecida
7,8,Brasilândia,243273,São Paulo,"(-21.255575, -52.0365617)",-21.255575,-52.036562,Zona Norte
8,9,Campo Limpo,236162,São Paulo,"(-23.6361999, -46.7627968)",-23.636200,-46.762797,Região desconhecida
9,10,Jabaquara,214958,São Paulo,"(-23.6463382, -46.6410462)",-23.646338,-46.641046,Região desconhecida
